In [1]:
import re
import math
import time
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2


In [ ]:
#PostgreSQLの接続情報
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'ja52712'
DB_HOST = 'localhost'
DB_PORT = '5432'

In [ ]:
#ウェブスクレイピングの練習サイトを利用
BASE = 'https://books.toscrape.com/'
HEADERS = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Scraping-Practice/1.0'
}

In [ ]:
#型ヒント（:str）を使用した関数parse_priceを定義
def parse_price(text:str):
    if not isinstance(text,str):#isinstance関数を使ってオブジェクトが指定した型かどうかを判定
        return None,None #戻り値がタプルなのでNoneを2つ設定
    symbol_map = {"£": "GBP", "€": "EUR", "$": "USD"}
    currency = None
    for sym,cur in symbol_map.items():
        if sym in text:
            currency = cur
            break
    m = re.search(r"\d+(?:\.\d+)?", text.replace(",", ""))
    return (float(m.group(0)),currency) if m else (None,currency)

In [5]:
def scrape_list_page(url:str):
    r = requests.get(url,headers=HEADERS,timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text,'html.parser')
    
    rows = []
    for li in soup.select('section div ol.row li'):
        a = li.select_one('h3 a')
        title = a.get('title')
        href = a.get('href')
        product_url = urljoin(url,href)
        
        price_text = li.select_one('.price_color').get_text(strip=True)
        price,currency = parse_price(price_text)
        
        stock_text = li.select_one('.availability').get_text(strip=True)
        
        raiting = None
        raiting_el = li.select_one('.star-rating')
        
        if raiting_el and raiting_el.has_attr('class'):
            for c in raiting_el['class']:
                if c in {"One","Two","Three","Four","Five"}:
                    raiting = c
                    
                    
        rows.append({
            "site": "books.toscrape",
            "title": title,
            "price": price,
            "currency": currency,
            "availability": stock_text,
            "rating": raiting,
            "url": product_url,
        })
        
        next_href = soup.select_one('li.next a')
        next_url = urljoin(url,next_href.get('href')) if next_href else None
        return rows,next_url

In [6]:
all_rows = []
url = BASE
pages = 0
MAX_PAGES = 5

while url and pages < MAX_PAGES:
    rows,url = scrape_list_page(url)
    all_rows.extend(rows)
    pages += 1
    time.sleep(0.8)
    
    
df = pd.DataFrame(all_rows)
display(df.head(10))
print('件数:',len(df))
print('欠損数:\n',df.isna().sum())

,site,title,price,currency,availability,rating,url
0,books.toscrape,A Light in the Attic,51.77,GBP,In stock,Three,https://books.toscrape.com/catalogue/a-light-i...
1,books.toscrape,In Her Wake,12.84,GBP,In stock,One,https://books.toscrape.com/catalogue/in-her-wa...
2,books.toscrape,Slow States of Collapse: Poems,57.31,GBP,In stock,Three,https://books.toscrape.com/catalogue/slow-stat...
3,books.toscrape,The Nameless City (The Nameless City #1),38.16,GBP,In stock,Four,https://books.toscrape.com/catalogue/the-namel...
4,books.toscrape,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,GBP,In stock,Five,https://books.toscrape.com/catalogue/princess-...


件数: 5
欠損数:
 site            0
title           0
price           0
currency        0
availability    0
rating          0
url             0
dtype: int64
